In [2]:
import os

In [15]:
%pwd

'd:\\Projects\\GIT'

In [ ]:
os.chdir('.././.')

In [21]:
os.chdir("GIT/Chicken_Disease_classification")

In [22]:
%pwd
os.listdir()

['.git',
 '.github',
 '.gitignore',
 'artifacts',
 'config',
 'data',
 'dvc.yaml',
 'github',
 'LICENSE',
 'logs',
 'main.py',
 'params.yaml',
 'README.md',
 'requirements.txt',
 'research',
 'scores.json',
 'setup.py',
 'src',
 'template.py',
 'templates',
 'test.py',
 'venv']

In [23]:
os.listdir()

['.git',
 '.github',
 '.gitignore',
 'artifacts',
 'config',
 'data',
 'dvc.yaml',
 'github',
 'LICENSE',
 'logs',
 'main.py',
 'params.yaml',
 'README.md',
 'requirements.txt',
 'research',
 'scores.json',
 'setup.py',
 'src',
 'template.py',
 'templates',
 'test.py',
 'venv']

In [24]:
import tensorflow as tf

In [25]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [26]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path: Path
    training_data_path: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [27]:
from src.Chicken_Disease_classification.constants import *
from src.Chicken_Disease_classification.utils.common import read_yaml, create_directory, save_json


In [28]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directory([self.config.artifacts_root]) 

    
    def get_validation_config(self) -> EvaluationConfig:

        eval_config = EvaluationConfig(
            model_path="artifacts/training/model.keras",
            training_data_path="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
    

In [29]:
from urllib.parse import urlparse

In [30]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config= config


    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data_path,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self) -> dict:
        self.model = self.load_model(self.config.model_path)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {
            "loss" : self.score[0],
            "accuracy" : self.score[1]
        }
        save_json(path=Path("scores.json"), data=scores)






In [31]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e


[2025-05-08 19:26:31,556] INFO: Successfully read YAML file 'Config\config.yaml'.
[2025-05-08 19:26:31,558] INFO: Successfully read YAML file 'params.yaml'.
[2025-05-08 19:26:31,558] INFO: Created directory at 'artifacts'.
Found 78 images belonging to 2 classes.


d:\Projects\GIT\Chicken_Disease_classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.9084 - loss: 1.1622
[2025-05-08 19:26:37,327] INFO: Successfully saved JSON file 'scores.json'.
